In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Load a Pretrained Model

In [2]:
### pretrained model options:
## templateGPT
# a 1m parameter model trained for 2000 iters with many layers, thin MLP hidden dimensions & few attention heads: 'models/templateGPT/trained/templateGPT_1m_tall_and_skinny'
# a 1m parameter model trained for 2000 iters with layers, MLP hidden dimensions, & attention heads bw the other two: 'models/templateGPT/trained/templateGPT_1m_5ft11_and_skinnyfat'
# a 1m parameter model trained for 2000 iters with few layers, thick MLP hidden dimensions & many attention heads: 'models/templateGPT/trained/templateGPT_1m_short_and_thicc'
# a 2m parameter model trained for 4000 iters with CosineNorm: 'models/templateGPT/trained/templateGPT_2m_CosineNorm'
# a 2m parameter model trained for 4000 iters with LayerNorm: 'models/templateGPT/trained/templateGPT_2m_LayerNorm'
# a 2m parameter model trained for 4000 iters with RMSNorm: 'models/templateGPT/trained/templateGPT_2m_RMSNorm'
# a 3m parameter model trained for 5000 iters with a gated MLP: 'models/templateGPT/trained/templateGPT_3m_GatedMLP'
# a 3m parameter model trained for 5000 iters with an old-fashioned MLP: 'models/templateGPT/trained/templateGPT_3m_GatedMLP'
# a 4m parameter model trained for 6000 iters with GeGLU activation: 'models/templateGPT/trained/templateGPT_4m_GeGLU'
# a 4m parameter model trained for 6000 iters with SwiGLU activation: 'models/templateGPT/trained/templateGPT_4m_SwiGLU'
## fractal-head-attention
# a 1m parameter model designed to be compared against the equivalent templateGPT: 'models/fractal-head-attention/trained/FHA_1m_short_and_thicc'
## Memory Mosaics
# a 1m parameter model designed to stick close to the paper's parameter ratios: 'models/MemoryMosaics/trained/MM_1m'
name = 'models/templateGPT/trained/templateGPT_4m_SwiGLU'

from tools import load_model
model, tokenizer, cfg = load_model(name)

# If you only plan to do inference, switch to evaluation mode
model.eval()

4011.456 K parameters 
 ModelConfig(dim=192, device='cpu', tokenizer='bpe_v1', vocab_len=8192, num_layers=6, second_resid_norm=False, mlp_hidden_mult=4, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=2, num_kv_heads=1, head_dim=96, theta=10000, max_seq_len=512, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1)


Model(
  (token_embedder): Embedding(8195, 192)
  (layers): ModuleList(
    (0-5): 6 x Layer(
      (pre_attn_norm): Norm()
      (attn): MQA(
        (Wq): Linear(in_features=192, out_features=192, bias=False)
        (Wk): Linear(in_features=192, out_features=96, bias=False)
        (Wv): Linear(in_features=192, out_features=96, bias=False)
        (Wo): Linear(in_features=192, out_features=192, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wup): Linear(in_features=192, out_features=512, bias=False)
        (Wgate): Linear(in_features=192, out_features=512, bias=False)
        (Wdown): Linear(in_features=512, out_features=192, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

# Inference

In [3]:
# importing this model's specific generate function
from tools import import_from_nested_path
path_parts = name.split('/')
imported_objects = import_from_nested_path([path_parts[0], path_parts[1]], 'inference', ['generate'])
generate = imported_objects.get('generate')

In [4]:
prompt = "Once upon a time, there was a boy named Tim. "
print(tokenizer.display(prompt))

Once| |upon| |a| |ti|me|, |th|er|e| |was| |a| |boy| |na|me|d| |Tim|. 


In [5]:
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 100,
    temperature = 0.7,
    memory_saver_div = 8,
    top_p = 0.9,
    top_k = 32,
)
print(output)

maximum attention matrix size in memory will be 64x512 rather than 512x512

Once upon a time, there was a boy named Tim. Tim loved to play with his toys and eat dirt. One day, Tim went to the park to play. He saw a big slide and wanted to climb it. Tim thought it would be fun to play with the top of the slide.
Tim had an idea. He would climb the slide and get it back to his room. He was very happy. Tim started to climb the slide and climb it. He climbed up the slide and got his new cake. But then, something unexpected happened. The slide started to blow down the slide! Tim and his friend were surprised and happy. The sun was not safe, and Tim was not happy again. He learned that he was not happy and always trying to try new things that can help us find the cake together.
